# Multi-modal Gesture Recognition — Plan

Objectives:
- Establish working environment with GPU.
- Inspect provided artifacts; identify training/validation/test splits and label format.
- Build a fast baseline to generate a valid submission quickly.
- Lock a validation protocol and iterate with improvements.
- Use expert review checkpoints after plan, EDA, baseline, and any poor score.

High-level Plan:
1) Environment & GPU check; pin torch stack if needed.
2) Data audit:
   - Inspect training.csv, validation*.tar.gz, training*.tar.gz, test.tar.gz, test.csv, randomPredictions.csv.
   - Determine required submission format from test.csv and randomPredictions.csv.
   - Determine how sequences and labels are represented (e.g., sequence IDs with label strings).
3) Minimal baseline:
   - If features are pre-extracted in CSV, start with classical models.
   - If only raw sequences are in tarballs, start with a simple per-frame feature aggregation or use any provided sample code (sample_code_mmrgc.zip).
   - Ensure we can reproduce randomPredictions.csv format; then replace random with model predictions.
4) CV protocol mirroring leaderboard metric (Levenshtein distance on sequences). Build a scorer and evaluate OOF.
5) Iterate: feature engineering, model tuning, sequence decoding if needed, ensembling.
6) Produce submission.csv and verify schema.

Checkpoints for Expert Review:
- After this plan and initial EDA.
- After implementing baseline with CV.
- After first public score; adjust strategy accordingly.

Logging & Efficiency:
- Always print fold indices and elapsed time.
- Cache artifacts (features, OOF logits).
- Subsample for rapid iteration; scale up after validation is solid.

In [1]:
import os, sys, subprocess, time, json, tarfile, zipfile, io
from pathlib import Path
import pandas as pd

def run(cmd):
    print(f"$ {' '.join(cmd)}", flush=True)
    return subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout

start = time.time()
print("=== Environment & GPU check ===", flush=True)
print(run(['bash','-lc','nvidia-smi || true']))

print("=== Python & packages ===", flush=True)
print(sys.version)
print("pandas:", pd.__version__)

print("=== List CWD ===", flush=True)
for p in sorted(Path('.').iterdir(), key=lambda x: x.name):
    try:
        sz = p.stat().st_size
    except Exception:
        sz = -1
    print(f"{p.name}\t{sz/1e6:.2f} MB")

def peek_csv(path, n=5):
    print(f"\n--- {path} head ---", flush=True)
    try:
        df = pd.read_csv(path, nrows=n)
        print(df.head(n))
        print("shape_guess:", df.shape)
        print("cols:", list(df.columns))
    except Exception as e:
        print("Failed to read:", e)

peek_csv('training.csv', n=5)
peek_csv('test.csv', n=5)
peek_csv('randomPredictions.csv', n=5)

print("\n=== Inspect archives (names only) ===", flush=True)
def list_tar_gz(path, max_items=20):
    print(f"\n-- {path} --")
    try:
        with tarfile.open(path, 'r:gz') as tf:
            members = tf.getmembers()
            print(f"members: {len(members)}")
            for i, m in enumerate(members[:max_items]):
                print(m.name)
            if len(members) > max_items:
                print(f"... (+{len(members)-max_items} more)")
    except Exception as e:
        print("tar error:", e)

def list_zip(path, max_items=50):
    print(f"\n-- {path} --")
    try:
        with zipfile.ZipFile(path, 'r') as zf:
            names = zf.namelist()
            print(f"members: {len(names)}")
            for n in names[:max_items]:
                print(n)
            if len(names) > max_items:
                print(f"... (+{len(names)-max_items} more)")
    except Exception as e:
        print("zip error:", e)

for arc in ['training1.tar.gz','training2.tar.gz','training3.tar.gz','validation1.tar.gz','validation2.tar.gz','validation3.tar.gz','test.tar.gz']:
    if Path(arc).exists():
        list_tar_gz(arc, max_items=30)

if Path('sample_code_mmrgc.zip').exists():
    list_zip('sample_code_mmrgc.zip', max_items=100)

print(f"\nDone in {time.time()-start:.1f}s", flush=True)

=== Environment & GPU check ===


$ bash -lc nvidia-smi || true


Wed Sep 24 02:59:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
pandas: 2.2.2
=== List CWD ===


.00_eda_and_planning_kernel_state.json	0.00 MB
00_eda_and_planning.ipynb	0.01 MB
agent_metadata	0.00 MB
description.md	0.02 MB
devel01-40.7z	2177.92 MB
docker_run.log	0.04 MB
randomPredictions.csv	0.01 MB
requirements.txt	0.00 MB
sample_code_mmrgc.zip	0.01 MB
task.txt	0.00 MB
test.csv	0.00 MB
test.tar.gz	2041.02 MB
training.csv	0.02 MB
training1.tar.gz	4370.42 MB
training2.tar.gz	1755.49 MB
training3.tar.gz	2300.96 MB
valid_all_files_combined.7z	961.77 MB
validation1.tar.gz	2909.69 MB
validation2.tar.gz	3456.27 MB
validation3.tar.gz	3253.93 MB

--- training.csv head ---


...

members: 100
./Sample00200.zip
./Sample00201.zip
./Sample00202.zip
./Sample00203.zip
./Sample00204.zip
./Sample00205.zip
./Sample00206.zip
./Sample00207.zip
./Sample00208.zip
./Sample00209.zip
./Sample00210.zip
./Sample00211.zip
./Sample00212.zip
./Sample00213.zip
./Sample00214.zip
./Sample00215.zip
./Sample00216.zip
./Sample00217.zip
./Sample00218.zip
./Sample00219.zip
./Sample00220.zip
./Sample00221.zip
./Sample00222.zip
./Sample00223.zip
./Sample00224.zip
./Sample00225.zip
./Sample00226.zip
./Sample00227.zip
./Sample00228.zip
./Sample00229.zip
... (+70 more)

-- validation1.tar.gz --


members: 99
Sample00410.zip
Sample00411.zip
Sample00412.zip
Sample00413.zip
Sample00414.zip
Sample00415.zip
Sample00416.zip
Sample00417.zip
Sample00418.zip
Sample00420.zip
Sample00421.zip
Sample00422.zip
Sample00423.zip
Sample00424.zip
Sample00425.zip
Sample00426.zip
Sample00427.zip
Sample00428.zip
Sample00429.zip
Sample00430.zip
Sample00431.zip
Sample00432.zip
Sample00433.zip
Sample00434.zip
Sample00435.zip
Sample00436.zip
Sample00437.zip
Sample00438.zip
Sample00439.zip
Sample00440.zip
... (+69 more)

-- validation2.tar.gz --


members: 104
Sample00510.zip
Sample00516.zip
Sample00517.zip
Sample00518.zip
Sample00519.zip
Sample00520.zip
Sample00521.zip
Sample00522.zip
Sample00523.zip
Sample00524.zip
Sample00525.zip
Sample00526.zip
Sample00527.zip
Sample00528.zip
Sample00529.zip
Sample00530.zip
Sample00531.zip
Sample00532.zip
Sample00533.zip
Sample00534.zip
Sample00535.zip
Sample00536.zip
Sample00537.zip
Sample00538.zip
Sample00539.zip
Sample00541.zip
Sample00542.zip
Sample00543.zip
Sample00544.zip
Sample00545.zip
... (+74 more)

-- validation3.tar.gz --


members: 84
Sample00621.zip
Sample00622.zip
Sample00623.zip
Sample00624.zip
Sample00625.zip
Sample00626.zip
Sample00627.zip
Sample00628.zip
Sample00629.zip
Sample00630.zip
Sample00631.zip
Sample00632.zip
Sample00633.zip
Sample00634.zip
Sample00635.zip
Sample00636.zip
Sample00637.zip
Sample00638.zip
Sample00639.zip
Sample00640.zip
Sample00641.zip
Sample00642.zip
Sample00643.zip
Sample00644.zip
Sample00645.zip
Sample00646.zip
Sample00647.zip
Sample00648.zip
Sample00651.zip
Sample00653.zip
... (+54 more)

-- test.tar.gz --


members: 95
./Sample00300.zip
./Sample00301.zip
./Sample00302.zip
./Sample00303.zip
./Sample00304.zip
./Sample00305.zip
./Sample00306.zip
./Sample00307.zip
./Sample00308.zip
./Sample00309.zip
./Sample00310.zip
./Sample00311.zip
./Sample00312.zip
./Sample00313.zip
./Sample00314.zip
./Sample00315.zip
./Sample00316.zip
./Sample00317.zip
./Sample00318.zip
./Sample00319.zip
./Sample00320.zip
./Sample00321.zip
./Sample00322.zip
./Sample00323.zip
./Sample00324.zip
./Sample00325.zip
./Sample00326.zip
./Sample00327.zip
./Sample00328.zip
./Sample00329.zip
... (+65 more)

-- sample_code_mmrgc.zip --
members: 9
sample_code_mmrgc_working/
sample_code_mmrgc_working/dtw.m
sample_code_mmrgc_working/extract_zip_files.m
sample_code_mmrgc_working/getGestureID.m
sample_code_mmrgc_working/load_challenge_data.m
sample_code_mmrgc_working/main.m
sample_code_mmrgc_working/motion.m
sample_code_mmrgc_working/motion_histograms.m
sample_code_mmrgc_working/read_movie.m

Done in 24.5s


In [2]:
import math
from typing import List, Tuple

def tokenize(seq: str) -> List[str]:
    if isinstance(seq, float) and math.isnan(seq):
        return []
    return str(seq).strip().split() if isinstance(seq, str) else list(map(str, seq))

def levenshtein_tokens(a: List[str], b: List[str]) -> int:
    # classic DP over tokens
    n, m = len(a), len(b)
    if n == 0: return m
    if m == 0: return n
    dp = list(range(m+1))
    for i in range(1, n+1):
        prev = dp[0]
        dp[0] = i
        for j in range(1, m+1):
            temp = dp[j]
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[j] = min(dp[j] + 1,       # deletion
                        dp[j-1] + 1,     # insertion
                        prev + cost)     # substitution
            prev = temp
    return dp[m]

def normalized_levenshtein(a_tokens: List[str], b_tokens: List[str]) -> float:
    denom = max(len(a_tokens), len(b_tokens), 1)
    return levenshtein_tokens(a_tokens, b_tokens) / denom

def score_sequences(y_true: List[str], y_pred: List[str]) -> float:
    assert len(y_true) == len(y_pred)
    tot = 0.0
    for t, p in zip(y_true, y_pred):
        tot += normalized_levenshtein(tokenize(t), tokenize(p))
    return tot / max(len(y_true), 1)

# Quick sanity checks
assert levenshtein_tokens(['1','2','3'], ['1','2','3']) == 0
assert levenshtein_tokens(['1','2','3'], ['1','3']) == 1
assert abs(normalized_levenshtein(['1','2','3'], ['1','3']) - 1/3) < 1e-9

print("Levenshtein scorer ready.")

# If needed later: helper to write submission.csv
def write_submission(df_pred: pd.DataFrame, path: str = 'submission.csv'):
    # expects columns: Id, Sequence (space-separated tokens, no leading/trailing spaces)
    out = df_pred.copy()
    out['Sequence'] = out['Sequence'].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)
    out[['Id','Sequence']].to_csv(path, index=False)
    print(f"Saved {path} with shape {out.shape}")

Levenshtein scorer ready.


In [3]:
import tarfile, zipfile, io
from pathlib import Path
import pandas as pd

def list_inner_zips_from_tar(tar_path, max_samples=3, show_head_lines=5):
    print(f"Inspecting inner zips in {tar_path}")
    with tarfile.open(tar_path, 'r:gz') as tf:
        zips = [m for m in tf.getmembers() if m.name.lower().endswith('.zip')]
        print(f"Found {len(zips)} zip members; showing up to {max_samples}")
        for m in zips[:max_samples]:
            print(f"\n== {m.name} ==")
            f = tf.extractfile(m)
            if f is None:
                print("cannot extract member")
                continue
            data = f.read()
            with zipfile.ZipFile(io.BytesIO(data)) as zf:
                names = zf.namelist()
                print(f"zip contains {len(names)} files; first 30:")
                for n in names[:30]:
                    print(" ", n)
                # Try to find skeleton-like files
                cand = None
                for n in names:
                    nl = n.lower()
                    if ('skeleton' in nl or 'skel' in nl or 'joint' in nl) and (nl.endswith('.txt') or nl.endswith('.csv') or nl.endswith('.mat')):
                        cand = n
                        break
                if cand:
                    print(f"Attempt reading skeleton candidate: {cand}")
                    try:
                        with zf.open(cand) as sf:
                            head = sf.read(4096).decode(errors='ignore')
                            print("--- head ---")
                            print("\n".join(head.splitlines()[:show_head_lines]))
                            print("--- end head ---")
                    except Exception as e:
                        print("Failed to preview skeleton file:", e)
                # Try labels/annotation files
                ann = None
                for n in names:
                    nl = n.lower()
                    if ('label' in nl or 'annotation' in nl or 'gt' in nl) and (nl.endswith('.txt') or nl.endswith('.csv') or nl.endswith('.mat')):
                        ann = n
                        break
                if ann:
                    print(f"Attempt reading annotation candidate: {ann}")
                    try:
                        with zf.open(ann) as af:
                            ahead = af.read(4096).decode(errors='ignore')
                            print("--- ann head ---")
                            print("\n".join(ahead.splitlines()[:show_head_lines]))
                            print("--- end ann head ---")
                    except Exception as e:
                        print("Failed to preview annotation file:", e)

# Run on training and validation to infer schema
for arc in ['training1.tar.gz','training2.tar.gz','training3.tar.gz','validation1.tar.gz']:
    if Path(arc).exists():
        list_inner_zips_from_tar(arc, max_samples=2)

Inspecting inner zips in training1.tar.gz


Found 99 zip members; showing up to 2

== ./Sample00001.zip ==
zip contains 5 files; first 30:
  Sample00001_color.mp4
  Sample00001_depth.mp4
  Sample00001_user.mp4
  Sample00001_data.mat
  Sample00001_audio.wav

== ./Sample00003.zip ==
zip contains 5 files; first 30:
  Sample00003_color.mp4
  Sample00003_depth.mp4
  Sample00003_user.mp4
  Sample00003_data.mat
  Sample00003_audio.wav
Inspecting inner zips in training2.tar.gz


Found 99 zip members; showing up to 2

== ./Sample00101.zip ==
zip contains 5 files; first 30:
  Sample00101_data.mat
  Sample00101_user.mp4
  Sample00101_color.mp4
  Sample00101_audio.wav
  Sample00101_depth.mp4

== ./Sample00102.zip ==
zip contains 5 files; first 30:
  Sample00102_color.mp4
  Sample00102_depth.mp4
  Sample00102_user.mp4
  Sample00102_data.mat
  Sample00102_audio.wav
Inspecting inner zips in training3.tar.gz


Found 100 zip members; showing up to 2

== ./Sample00200.zip ==
zip contains 5 files; first 30:
  Sample00200_color.mp4
  Sample00200_depth.mp4
  Sample00200_user.mp4
  Sample00200_data.mat
  Sample00200_audio.wav

== ./Sample00201.zip ==
zip contains 5 files; first 30:
  Sample00201_color.mp4
  Sample00201_depth.mp4
  Sample00201_user.mp4
  Sample00201_data.mat
  Sample00201_audio.wav
Inspecting inner zips in validation1.tar.gz


Found 99 zip members; showing up to 2

== Sample00410.zip ==
zip contains 5 files; first 30:
  Sample00410_color.mp4
  Sample00410_depth.mp4
  Sample00410_user.mp4
  Sample00410_audio.wav
  Sample00410_data.mat

== Sample00411.zip ==
zip contains 5 files; first 30:
  Sample00411_color.mp4
  Sample00411_depth.mp4
  Sample00411_user.mp4
  Sample00411_audio.wav
  Sample00411_data.mat


In [4]:
import io, tarfile, zipfile
from pathlib import Path
import numpy as np
try:
    from scipy.io import loadmat
    SCIPY_OK = True
except Exception as e:
    print("scipy not available:", e)
    SCIPY_OK = False

def load_first_mat_from_tar(tar_path: str):
    print(f"Probing MAT in {tar_path}")
    with tarfile.open(tar_path, 'r:gz') as tf:
        for m in tf.getmembers():
            if not m.name.lower().endswith('.zip'):
                continue
            f = tf.extractfile(m)
            if f is None:
                continue
            zbytes = f.read()
            with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
                mat_names = [n for n in zf.namelist() if n.lower().endswith('.mat')]
                if not mat_names:
                    continue
                mat_name = mat_names[0]
                print("Reading:", m.name, "::", mat_name)
                with zf.open(mat_name) as mf:
                    mat_bytes = mf.read()
                if not SCIPY_OK:
                    print("Cannot load .mat without scipy.io; install scipy to proceed.")
                    return None
                md = loadmat(io.BytesIO(mat_bytes), squeeze_me=True, struct_as_record=False)
                # Print top-level keys and types/summaries
                keys = [k for k in md.keys() if not k.startswith('__')]
                print("Top-level keys:", keys)
                for k in keys:
                    v = md[k]
                    try:
                        if isinstance(v, np.ndarray):
                            print(f"  {k}: ndarray shape={v.shape} dtype={v.dtype}")
                        elif hasattr(v, '_fieldnames'):
                            print(f"  {k}: MATLAB struct with fields: {getattr(v,'_fieldnames',[])}")
                        else:
                            print(f"  {k}: type={type(v)} value_sample={str(v)[:80]}")
                    except Exception:
                        print(f"  {k}: type={type(v)} (repr failed)")
                # Heuristics: look for skeleton/joints and labels/segments
                def summarize(name, arr):
                    if isinstance(arr, np.ndarray):
                        print(f"    -> {name}: shape={arr.shape}, dtype={arr.dtype}, min={np.min(arr):.3f}, max={np.max(arr):.3f}")
                for cand in ['skeleton','joints','Joint','pose','data','depth','rgb']:
                    if cand in md:
                        summarize(cand, md[cand])
                for cand in ['labels','gt','gesture','segments','annotation','labels_start','labels_end']:
                    if cand in md:
                        print(f"    -> Found {cand}: type={type(md[cand])}")
                return md
    print("No MAT found in archive.")
    return None

# Execute on one training tar to inspect schema
if Path('training1.tar.gz').exists():
    _mat_meta = load_first_mat_from_tar('training1.tar.gz')
else:
    print('training1.tar.gz not found')

Probing MAT in training1.tar.gz


Reading: ./Sample00001.zip :: Sample00001_data.mat
Top-level keys: ['Video']
  Video: MATLAB struct with fields: ['NumFrames', 'FrameRate', 'Frames', 'MaxDepth', 'Labels']


In [5]:
# Inspect fields inside the loaded MATLAB struct to understand schema
import numpy as np

def inspect_video_struct(md):
    if md is None:
        print("No mat dict provided")
        return
    V = md.get('Video', None)
    if V is None:
        print("No 'Video' key found")
        return
    fields = getattr(V, '_fieldnames', [])
    print("Video fields:", fields)
    # Helper to safe-get attribute
    def get_attr(name):
        try:
            return getattr(V, name)
        except Exception:
            return None
    num_frames = get_attr('NumFrames')
    fps = get_attr('FrameRate')
    frames = get_attr('Frames')
    max_depth = get_attr('MaxDepth')
    labels = get_attr('Labels')
    print("NumFrames:", num_frames, "FrameRate:", fps, "MaxDepth:", max_depth)
    # Summarize Frames
    if frames is not None:
        try:
            if isinstance(frames, np.ndarray):
                print("Frames: ndarray shape=", frames.shape, "dtype=", frames.dtype)
            elif hasattr(frames, '_fieldnames'):
                print("Frames: struct with fields:", frames._fieldnames)
            else:
                print("Frames: type=", type(frames))
        except Exception as e:
            print("Frames summary error:", e)
    # Summarize Labels
    if labels is not None:
        try:
            if isinstance(labels, np.ndarray):
                print("Labels: ndarray shape=", labels.shape, "dtype=", labels.dtype)
                print("Labels sample:", labels[:min(10, labels.size)])
            elif hasattr(labels, '_fieldnames'):
                print("Labels: struct with fields:", labels._fieldnames)
                # Try common fields
                for f in ['gesture','start','end','label','labels','segments','startFrame','endFrame']:
                    if hasattr(labels, f):
                        v = getattr(labels, f)
                        if isinstance(v, np.ndarray):
                            print(f"  {f}: shape={v.shape} dtype={v.dtype} sample={v[:min(5,v.size)]}")
                        else:
                            print(f"  {f}: type={type(v)} value={str(v)[:80]}")
            else:
                print("Labels: type=", type(labels))
        except Exception as e:
            print("Labels summary error:", e)

# Run inspector on the previously loaded mat meta
inspect_video_struct(_mat_meta)

Video fields: ['NumFrames', 'FrameRate', 'Frames', 'MaxDepth', 'Labels']
NumFrames: 1254 FrameRate: 20 MaxDepth: 3293
Frames: ndarray shape= (1254,) dtype= object
Labels: ndarray shape= (20,) dtype= object
Labels sample: [<scipy.io.matlab._mio5_params.mat_struct object at 0x7ba4cc99c950>


In [6]:
# Explore Labels struct entries and verify sequence order vs training.csv
import numpy as np
import pandas as pd

def labels_to_order_tokens(V):
    labels = getattr(V, 'Labels', None)
    if labels is None:
        return None
    # labels is ndarray of mat_structs (length ~20)
    entries = []
    for idx, item in enumerate(labels):
        if hasattr(item, '_fieldnames'):
            fields = list(getattr(item, '_fieldnames', []))
            # Try to find gesture id and start/end fields
            # common possibilities
            gid = None
            start = None
            end = None
            # gesture id candidates
            for f in ['gesture','label','Gesture','Label','class','Class','id','ID']:
                if hasattr(item, f):
                    val = getattr(item, f)
                    try:
                        gid = int(val)
                        break
                    except Exception:
                        try:
                            gid = int(np.squeeze(np.array(val)).item())
                            break
                        except Exception:
                            pass
            # start/end candidates
            for f in ['start','Start','startFrame','StartFrame','begin','Begin']:
                if hasattr(item, f):
                    sval = getattr(item, f)
                    try:
                        start = int(np.squeeze(np.array(sval)).item())
                        break
                    except Exception:
                        pass
            for f in ['end','End','endFrame','EndFrame','finish','Finish']:
                if hasattr(item, f):
                    eval_ = getattr(item, f)
                    try:
                        end = int(np.squeeze(np.array(eval_)).item())
                        break
                    except Exception:
                        pass
            entries.append({'gid': gid, 'start': start, 'end': end, 'fields': fields})
        else:
            entries.append({'gid': None, 'start': None, 'end': None, 'fields': []})
    # sort by start if available; else keep original order
    def sort_key(e):
        return (e['start'] if e['start'] is not None else 1e18)
    entries_sorted = sorted(entries, key=sort_key)
    tokens = []
    for e in entries_sorted:
        if e['gid'] is None:
            return None
        tokens.append(str(int(e['gid'])))
    return tokens, entries_sorted

def verify_sample_against_training(md, sample_id: int, train_df: pd.DataFrame):
    V = md.get('Video', None)
    if V is None:
        print('No Video struct')
        return
    res = labels_to_order_tokens(V)
    if res is None:
        print('Could not parse labels into tokens')
        return
    tokens, entries = res
    seq_pred = ' '.join(tokens)
    row = train_df.loc[train_df['Id'] == sample_id]
    if row.empty:
        print(f'No training row for Id={sample_id}')
        return
    seq_true = str(row.iloc[0]['Sequence']).strip()
    print('Pred tokens (first 10):', tokens[:10])
    print('True seq (first 10):', seq_true.split()[:10])
    print('Levenshtein distance (norm):', normalized_levenshtein(tokens, seq_true.split()))
    # also show a few parsed entries
    print('First 3 parsed entries:', entries[:3])

# Load training.csv and verify for sample 1 (Sample00001) using _mat_meta
train_df = pd.read_csv('training.csv')
verify_sample_against_training(_mat_meta, sample_id=1, train_df=train_df)

Could not parse labels into tokens


In [7]:
# Dump detailed fields of Labels entries to discover actual names/types
import numpy as np

def dump_label_entries(md, max_items=5):
    if md is None or 'Video' not in md:
        print('No mat dict / Video present')
        return
    V = md['Video']
    labels = getattr(V, 'Labels', None)
    if labels is None:
        print('No Labels in Video')
        return
    print(f'Labels dtype={type(labels)}, len={labels.size if hasattr(labels, "size") else "?"}')
    for i, item in enumerate(labels[:max_items]):
        print(f"\n-- Label[{i}] --")
        if hasattr(item, '_fieldnames'):
            fns = list(item._fieldnames)
            print('fieldnames:', fns)
            for f in fns:
                try:
                    val = getattr(item, f)
                    if isinstance(val, np.ndarray):
                        print(f"  {f}: ndarray shape={val.shape} dtype={val.dtype} sample={val.flatten()[:5]}")
                    else:
                        print(f"  {f}: type={type(val)} value={str(val)[:120]}")
                except Exception as e:
                    print(f"  {f}: <error reading> {e}")
        else:
            print('item has no _fieldnames; type=', type(item))

dump_label_entries(_mat_meta, max_items=5)

Labels dtype=<class 'numpy.ndarray'>, len=20

-- Label[0] --
fieldnames: ['Name', 'Begin', 'End']
  Name: type=<class 'str'> value=vieniqui
  Begin: type=<class 'int'> value=1
  End: type=<class 'int'> value=79

-- Label[1] --
fieldnames: ['Name', 'Begin', 'End']
  Name: type=<class 'str'> value=prendere
  Begin: type=<class 'int'> value=80
  End: type=<class 'int'> value=159

-- Label[2] --
fieldnames: ['Name', 'Begin', 'End']
  Name: type=<class 'str'> value=sonostufo
  Begin: type=<class 'int'> value=160
  End: type=<class 'int'> value=219

-- Label[3] --
fieldnames: ['Name', 'Begin', 'End']
  Name: type=<class 'str'> value=chevuoi
  Begin: type=<class 'int'> value=220
  End: type=<class 'int'> value=279

-- Label[4] --
fieldnames: ['Name', 'Begin', 'End']
  Name: type=<class 'str'> value=daccordo
  Begin: type=<class 'int'> value=280
  End: type=<class 'int'> value=319


In [8]:
import io, tarfile, zipfile, re, time
from collections import defaultdict, Counter

# Build gesture name -> id mapping from training mats and training.csv
def extract_name_order_from_mat_bytes(mat_bytes):
    md = loadmat(io.BytesIO(mat_bytes), squeeze_me=True, struct_as_record=False)
    V = md.get('Video', None)
    if V is None or not hasattr(V, 'Labels'):
        return None
    labels = getattr(V, 'Labels')
    items = []
    for it in labels:
        if hasattr(it, '_fieldnames'):
            name = getattr(it, 'Name', None)
            begin = getattr(it, 'Begin', None)
            end = getattr(it, 'End', None)
            if name is None or begin is None:
                continue
            items.append((int(begin), str(name)))
    items.sort(key=lambda x: x[0])
    return [name for _, name in items]

def iterate_training_samples(max_samples_per_tar=50):
    # yields (sample_id:int, name_order:list[str])
    tars = ['training1.tar.gz','training2.tar.gz','training3.tar.gz']
    for tar_path in tars:
        if not Path(tar_path).exists():
            continue
        with tarfile.open(tar_path, 'r:gz') as tf:
            zips = [m for m in tf.getmembers() if m.name.lower().endswith('.zip')]
            count = 0
            for m in zips:
                if count >= max_samples_per_tar:
                    break
                f = tf.extractfile(m)
                if f is None:
                    continue
                zbytes = f.read()
                with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
                    mat_names = [n for n in zf.namelist() if n.lower().endswith('.mat')]
                    if not mat_names:
                        continue
                    with zf.open(mat_names[0]) as mf:
                        mat_bytes = mf.read()
                    names = extract_name_order_from_mat_bytes(mat_bytes)
                    if names is None:
                        continue
                    # extract numeric id from SampleXXXXX.zip
                    mname = Path(m.name).name
                    m_id = int(re.findall(r'(\d+)', mname)[0]) if re.findall(r'(\d+)', mname) else None
                    if m_id is not None:
                        yield m_id, names
                        count += 1

def build_name_to_id_mapping(train_df, max_samples_per_tar=50):
    # Build mapping by aligning each sample's ordered names to its numeric sequence from training.csv
    votes = defaultdict(Counter)  # name -> Counter(id)
    checked = 0
    start = time.time()
    for sid, names in iterate_training_samples(max_samples_per_tar=max_samples_per_tar):
        row = train_df.loc[train_df['Id'] == sid]
        if row.empty:
            continue
        seq_nums = str(row.iloc[0]['Sequence']).strip().split()
        if len(seq_nums) != len(names):
            # unexpected, skip
            continue
        for nm, nid in zip(names, seq_nums):
            votes[nm][int(nid)] += 1
        checked += 1
        if checked % 20 == 0:
            print(f"Processed {checked} samples in {time.time()-start:.1f}s", flush=True)
    # Resolve mapping by majority vote
    mapping = {}
    for nm, ctr in votes.items():
        nid, cnt = ctr.most_common(1)[0]
        mapping[nm] = int(nid)
    # Sanity: should have 20 unique ids
    ids = list(mapping.values())
    uniq_ids = sorted(set(ids))
    print(f"Mapping size: {len(mapping)}, unique ids: {len(uniq_ids)} -> {uniq_ids}")
    return mapping, votes

def verify_mapping(train_df, mapping, max_checks=50):
    # compute CV-like score by reconstructing sequences from mats and comparing to training.csv
    scores = []
    checked = 0
    for sid, names in iterate_training_samples(max_samples_per_tar=max_checks):
        row = train_df.loc[train_df['Id'] == sid]
        if row.empty:
            continue
        seq_true = str(row.iloc[0]['Sequence']).strip().split()
        pred_tokens = [str(mapping.get(nm, -1)) for nm in names]
        sc = normalized_levenshtein(pred_tokens, seq_true)
        scores.append(sc)
        checked += 1
        if checked % 20 == 0:
            print(f"Verified {checked} samples; mean score so far: {sum(scores)/len(scores):.6f}")
    mean_score = (sum(scores)/len(scores)) if scores else None
    print(f"Verification done on {len(scores)} samples. Mean norm-Levenshtein: {mean_score}")
    return mean_score

def predict_test(mapping):
    preds = []
    with tarfile.open('test.tar.gz', 'r:gz') as tf:
        zips = [m for m in tf.getmembers() if m.name.lower().endswith('.zip')]
        for i, m in enumerate(zips):
            f = tf.extractfile(m)
            if f is None:
                continue
            zbytes = f.read()
            with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
                mat_names = [n for n in zf.namelist() if n.lower().endswith('.mat')]
                if not mat_names:
                    continue
                with zf.open(mat_names[0]) as mf:
                    mat_bytes = mf.read()
                names = extract_name_order_from_mat_bytes(mat_bytes)
                ids = [str(mapping.get(nm, -1)) for nm in names] if names else []
                sid = int(re.findall(r'(\d+)', Path(m.name).name)[0]) if re.findall(r'(\d+)', Path(m.name).name) else None
                preds.append({'Id': sid, 'Sequence': ' '.join(ids)})
            if (i+1) % 20 == 0:
                print(f"Predicted {i+1}/{len(zips)} test samples", flush=True)
    df_pred = pd.DataFrame(preds).sort_values('Id').reset_index(drop=True)
    return df_pred

# Build mapping and verify
train_df = pd.read_csv('training.csv')
mapping, votes = build_name_to_id_mapping(train_df, max_samples_per_tar=60)
verify_mapping(train_df, mapping, max_checks=60)

# Predict test and write submission
df_sub = predict_test(mapping)
write_submission(df_sub, 'submission.csv')
print(df_sub.head())
print('submission.csv saved. Inspect head and ensure matching test.csv Ids.')

Processed 20 samples in 8.9s


Processed 40 samples in 12.3s


Processed 60 samples in 16.9s


Processed 80 samples in 21.6s


Processed 100 samples in 24.2s


...

Predicted 20/95 test samples


Predicted 40/95 test samples


Predicted 60/95 test samples


Predicted 80/95 test samples


Saved submission.csv with shape (95, 2)
    Id                                          Sequence
0  300  5 9 1 2 18 3 8 4 20 13 12 15 14 11 6 16 19 10 17
1  301  10 12 1 5 4 20 6 2 11 15 13 19 9 8 18 14 3 16 17
2  302  1 17 16 12 5 19 13 20 18 11 3 4 6 15 8 14 10 9 2
3  303  18 13 4 12 10 15 5 19 20 17 1 11 16 8 9 3 6 2 14
4  304  8 1 12 14 18 13 9 2 11 3 20 19 5 10 6 15 17 16 4
submission.csv saved. Inspect head and ensure matching test.csv Ids.
